In [ ]:
!wget https://raw.githubusercontent.com/openforcefield/2023-workshop-vignettes/main/colab_setup.ipynb
%run colab_setup.ipynb
%env LD_LIBRARY_PATH=$LD_LIBRARY_PATH:$CONDA_PREFIX/lib/

The starting configuration for this vignette comes from the Toolkit Showcase, which assembles and solvates this topology starting from a prepared protein PDB and ligand SDF. 

https://github.com/openforcefield/openff-toolkit/tree/main/examples/toolkit_showcase

# OpenFF Code

In [3]:
from openff.toolkit import Molecule, Topology, ForceField

ligand = Molecule.from_file("inputs/PT2385.sdf")
top = Topology.from_pdb("inputs/solvated_complex.pdb", unique_molecules=[ligand])
ff = ForceField("openff-2.0.0.offxml", "ff14sb_off_impropers_0.0.3.offxml")
sys = ff.create_openmm_system(top)

# OpenMM Code

In [5]:
import openmm
from openff.units import Quantity, unit
from openmm import unit as openmm_unit

# Construct and configure a Langevin integrator at 300 K with an appropriate friction constant and time-step
integrator = openmm.LangevinIntegrator(
    300 * openmm_unit.kelvin,
    1 / openmm_unit.picosecond,
    0.002 * openmm_unit.picoseconds,
)

# Combine the topology, system, integrator and initial positions into a simulation
simulation = openmm.app.Simulation(top.to_openmm(), sys, integrator)
simulation.context.setPositions(top.get_positions().to_openmm())

# Add a reporter to record the structure every 10 steps
dcd_reporter = openmm.app.DCDReporter("trajectory.dcd", 250)
simulation.reporters.append(dcd_reporter)
simulation.context.setVelocitiesToTemperature(300 * openmm_unit.kelvin)
simulation.runForClockTime(1.5 * openmm_unit.minute)

# NGLView

In [7]:
import mdtraj
import nglview

trajectory: mdtraj.Trajectory = mdtraj.load(
    "trajectory.dcd", top=mdtraj.Topology.from_openmm(top.to_openmm())
)

view = nglview.show_mdtraj(trajectory.image_molecules())
view.add_representation("line", selection="water")
view.add_representation(
    "hyperball", radiusSize=1, radiusScale=0.5, selection="not protein and not water"
)
view

NGLWidget(max_frame=41)